In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pysa.emd as emddev
import pysa.eemd as eemddev
import pysa.visualization as plotter
import pysa.utils as utils
import pysa.nhht as nhht
import copy
from multiprocessing import Pool
from timeit import default_timer as timer

from scipy import signal
import scipy
import os
from scipy import fft
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.cross_validation import train_test_split
%matplotlib inline
test = 1
plt.rcParams['figure.figsize'] = (12,8)

In [7]:
def classify_SVM(feat_vectors):
    train = np.vstack(tuple(feat_vectors))
    X, Y = train[:, 0:-1], train[:, -1].astype(int)
    clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(kernel='rbf'))
    scores = cross_val_score(clf, X, Y, cv=3, scoring='accuracy')
    print("Accuracy: ", scores.mean())

In [119]:
from pprint import pprint
def grid_search_SVM(X_train, Y_train):
    
    tuned_parameters2 = [{'kernel': ['rbf'], 'gamma': [0.0001,0.001,0.01,1.0,2],
                         'C': [1,2,4,6,8,10]},
                        {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
    
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1, 0.1, 1e-2, 1e-3, 1e-4, 1e-5],
                         'C': [0.01, 0.1, 1, 10, 100, 1000, 10000]},
                        {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
    scaler = StandardScaler()
    scaler.fit(X_train)

    grid = GridSearchCV(estimator=svm.SVC(C=1), param_grid=tuned_parameters, cv=4, scoring='accuracy', n_jobs=-1) 
    grid.fit(scaler.transform(X_train), Y_train)

    # View the accuracy score
    print('Best score for data1:', grid.best_score_)
    # View the best parameters for the model found using grid search
    print('Best C:',grid.best_estimator_.C) 
    print('Best Kernel:',grid.best_estimator_.kernel)
    print('Best Gamma:',grid.best_estimator_.gamma)
    pprint(grid.grid_scores_)
    return grid

In [120]:
path = "exp_data/joachim/"

In [121]:
feat_vec1 = np.load(path + 'feat_vec1.npy')
feat_vec2 = np.load(path + 'feat_vec2.npy')
feat_vec3 = np.load(path + 'feat_vec3.npy')

In [122]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(
    feat_vec1[:,:-1], feat_vec1[:,-1], test_size=0.2, random_state=42
)
X_train2, X_test2, y_train2, y_test2 = train_test_split(
    feat_vec2[:,:-1], feat_vec2[:,-1], test_size=0.2, random_state=42
)
X_train3, X_test3, y_train3, y_test3 = train_test_split(
    feat_vec3[:,:-1], feat_vec3[:,-1], test_size=0.2, random_state=42
)

In [123]:
X_train = np.vstack(tuple([X_train1, X_train2]))
y_train = np.append(y_train1, y_train2)
X_test = np.vstack(tuple([X_test1, X_test2]))
y_test = np.append(y_test1, y_test2)

In [124]:
y_train

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.])

In [125]:
grid = grid_search_SVM(X_train, y_train)

Best score for data1: 0.5625
Best C: 0.01
Best Kernel: rbf
Best Gamma: 0.1
[mean: 0.42500, std: 0.04330, params: {'C': 0.01, 'gamma': 1, 'kernel': 'rbf'},
 mean: 0.56250, std: 0.10825, params: {'C': 0.01, 'gamma': 0.1, 'kernel': 'rbf'},
 mean: 0.47500, std: 0.02500, params: {'C': 0.01, 'gamma': 0.01, 'kernel': 'rbf'},
 mean: 0.47500, std: 0.04330, params: {'C': 0.01, 'gamma': 0.001, 'kernel': 'rbf'},
 mean: 0.46250, std: 0.06495, params: {'C': 0.01, 'gamma': 0.0001, 'kernel': 'rbf'},
 mean: 0.46250, std: 0.06495, params: {'C': 0.01, 'gamma': 1e-05, 'kernel': 'rbf'},
 mean: 0.42500, std: 0.04330, params: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'},
 mean: 0.56250, std: 0.10825, params: {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'},
 mean: 0.47500, std: 0.02500, params: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'},
 mean: 0.47500, std: 0.04330, params: {'C': 0.1, 'gamma': 0.001, 'kernel': 'rbf'},
 mean: 0.46250, std: 0.06495, params: {'C': 0.1, 'gamma': 0.0001, 'kernel': 'rbf'},
 mean: 0.46250, st

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [126]:
y_pred = grid.predict(X_test)

In [130]:
y_pred

array([ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.])

In [128]:
y_test

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.])

In [129]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

        1.0       0.00      0.00      0.00        10
        2.0       0.50      1.00      0.67        10

avg / total       0.25      0.50      0.33        20



/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
